# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Baehr  ->  H. Baehr  |  ['H. Baehr']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
T. Henning  ->  T. Henning  |  ['T. Henning']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 30 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.13246


extracting tarball to tmp_2504.13246...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.13377


extracting tarball to tmp_2504.13377...

...

T. Henning  ->  T. Henning  |  ['T. Henning']


Found 65 bibliographic references in tmp_2504.13679/HD106906_disk.bbl.
Retrieving document from  https://arxiv.org/e-print/2504.13766


extracting tarball to tmp_2504.13766...

 done.
  0: tmp_2504.13766/natnotes.tex, 332 lines
  1: tmp_2504.13766/natbib.tex, 96 lines
  2: tmp_2504.13766/aassymbols.tex, 579 lines
  3: tmp_2504.13766/sample631.tex, 1,421 lines


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2504.13766/sample631.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.13377-b31b1b.svg)](https://arxiv.org/abs/2504.13377) | **JWST's sharper view of EX Lup: cold water from ice sublimation during accretion outbursts**  |
|| S. A. Smith, et al. -- incl., <mark>A. Kospal</mark>, <mark>J. Bouwman</mark> |
|*Appeared on*| *2025-04-21*|
|*Comments*| *Accepted for publication in ApJ Letters*|
|**Abstract**|            The unstable accretion phases during pre-main-sequence evolution of T Tauri stars produce variable irradiation and heating of planet-forming regions. A strong accretion outburst was observed with Spitzer-IRS in 2008 in EX Lup, the prototype of EXor variables, and found to increase the mid-infrared water and OH emission and decrease organic emission, suggesting large chemical changes. We present here two JWST-MIRI epochs of quiescent EX Lup in 2022 and 2023 obtained over a decade after the 2008 outburst and several months after a moderate burst in 2022. With JWST's sharper spectral view, we can now analyze water emission as a function of temperature in the two MIRI epochs and, approximately, also in the previous Spitzer epochs. This new analysis shows a strong cold water vapor ``burst" in low-energy lines during the 2008 outburst, which we consider clear evidence for enhanced ice sublimation due to a recession of the snowline, as found in protostellar envelopes. JWST shows that EX Lup still has an unusually strong emission from cold water in comparison to other T Tauri disks, suggesting > 10-yr-long freeze-out timescales in the inner disk surface. EX Lup demonstrates that outbursts can significantly change the observed organic-to-water ratios and increase the cold water reservoir, providing chemical signatures to study the recent accretion history of disks. This study provides an unprecedented demonstration of the chemical evolution triggered by accretion outbursts in the Class II phase and of the high potential of time-domain experiments to reveal processes that may have fundamental implications on planet-forming bodies near the snowline.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.13679-b31b1b.svg)](https://arxiv.org/abs/2504.13679) | **MIRI-JWST mid-infrared direct imaging of the debris disk of HD106906**  |
|| D. Rouan, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-04-21*|
|*Comments*| **|
|**Abstract**|            We report MIRI-JWST coronagraphic observations at 11.3 and 15.5 mic of the debris disk around the young star HD 106906. The observations were made to characterize the structure, temperature and mass of the disk through the thermal emission of the dust heated by the central star. Another goal was also to constrain the size distribution of the grains. The data were reduced and calibrated using the JWST pipeline. The analysis was based on a forward-modeling of the images using a multiparameter radiative transfer model coupled to an optical code for coronagraphy processing. The disk is clearly detected at both wavelengths. The slight asymmetry is geometrically consistent with the asymmetry observed in the near-IR, but it is inconsistent the brightness distribution. The observed structure is well reproduced with a model of a disk (or belt) with a critical radius 70 au, a mildly inward-increasing density (index 2) and a steeper decrease outward (index -6). This indication of a filled disk inside the critical radius is inconsistent with sculpting from an inner massive planet. The size distribution of the grains that cause the mid-IR emission is well constrained by the flux ratio at the two wavelengths : 0.45 10 mic and 0.65 10 mic for silicate and graphite grains, respectively. The minimum size is consistent with predictions of blowout through radiative pressure. We derive a mass of the dust that causes the mid-IR emission of 3.3 5.0 E3 Mearth. When the larger grains (up to 1 cm) that cause the millimeter emission are included, we extrapolate this mass to 0.10 0.16 Mearth. We point out to that this is fully consistent with ALMA observations of the disk in terms of dust mass and of its millimeter flux. We estimate the average dust temperature in the planetesimal belt to be 74 K, and a temperature range within the whole disk from 40 to 130 K.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.13246-b31b1b.svg)](https://arxiv.org/abs/2504.13246) | **Turbulence Inhibits Planetesimal Formation in Class 0/I Disks Subject to Infall**  |
|| D. Carrera, et al. -- incl., <mark>H. Baehr</mark> |
|*Appeared on*| *2025-04-21*|
|*Comments*| *20 pages, 13 figures, submitted to ApJ*|
|**Abstract**|            There is growing evidence that planet formation begins early, within the $\lesssim 1$Myr Class 0/I phase, when infall dominates disk dynamics. Our goal is to determine if Class 0/I disks reach the conditions needed to form planetesimals ($\sim 100$km planet building blocks) by the streaming instability (SI). We focus on a recent suggestion that early infall causes an ``inflationary'' phase in which dust grains are advected outward. We modified the \texttt{DustPy} code to build a 1D disk that includes dust evolution, infall, and heating and cooling sources. We ran six models and examined the implications for the SI, taking into account recent works on how the SI responds to external turbulence. In line with other works, we find that grains are advected outward, which leads to ``advection-condensation-drift'' loop that greatly enhances the dust density at the water snowline. However, we do not see this process at the silicate line. Instead, we find a new pile up at the edge of the expanding disk. However, despite these localized enhancements, even a modest amount of turbulence ($\alpha = 10^{-3}$) leaves planetesimal formation far out of reach. The midplane dust-to-gas ratio is at least an order of magnitude below the SI threshold, even taking into account recent results on how dust coagulation boosts the SI. For planetesimals to form in the Class 0/I phase may require a way to transport angular momentum without turbulence (e.g., disk winds) or a non-SI mechanism to form planetesimals.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.13766-b31b1b.svg)](https://arxiv.org/abs/2504.13766) | **The impact of new ($α$, n) reaction rates on the weak s-process in metal-poor massive stars**  |
|| W. Xin, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-21*|
|*Comments*| *16 pages, 15 figures*|
|**Abstract**|            Massive stars are significant sites for the weak s-process (ws-process). $^{22}$Ne and $^{16}$O are, respectively, the main neutron source and poison for the ws-process. In the metal-poor stars, the abundance of $^{22}$Ne is limited by the metallicity, so that the contribution of $^{22}$Ne($\alpha$, n)$^{25}$Mg reaction on the s-process is small. Conversely, the $^{17}$O($\alpha$, n)$^{20}$Ne reaction is more evident in more metal-poor stars due to the most abundant $^{16}$O in all metallicities. In this work, we calculate the evolution of four metal-poor models ($Z=10^{-3}$) for the Zero-Age Main-Sequence (ZAMS) masses of $M ({\rm ZAMS})=$ 15, 20, 25, and 30 M$_{\odot}$ to investigate the effect of reaction rates on the ws-process. We adopt the new $^{17}$O($\alpha$, n)$^{20}$Ne and $^{17}$O($\alpha, \gamma$)$^{21}$Ne reaction rates suggested by Wiescher et al. (2023) and $^{22}$Ne($\alpha$, n)$^{25}$Mg and $^{22}$Ne($\alpha, \gamma$)$^{26}$Mg from Best et al. (2013). The yields of the s-process isotope with updated reaction rates are compared with the results using default reaction rates from JINA REACLIB. We find that the effects of new $^{17}$O+$\alpha$ are much more significant than those of new $^{22}$Ne+$\alpha$ reaction rates in the non-rotation stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.13377/./EXLup_23umlines_ratios.png', 'tmp_2504.13377/./all_spectra.png', 'tmp_2504.13377/./EXLup_diagnostic_diagram.png']
copying  tmp_2504.13377/./EXLup_23umlines_ratios.png to _build/html/
copying  tmp_2504.13377/./all_spectra.png to _build/html/
copying  tmp_2504.13377/./EXLup_diagnostic_diagram.png to _build/html/
exported in  _build/html/2504.13377.md
    + _build/html/tmp_2504.13377/./EXLup_23umlines_ratios.png
    + _build/html/tmp_2504.13377/./all_spectra.png
    + _build/html/tmp_2504.13377/./EXLup_diagnostic_diagram.png
found figures ['tmp_2504.13679/./F15F11_Sil.png', 'tmp_2504.13679/./HD106906_fig1_paper.png', 'tmp_2504.13679/./HD106906_fig2_contours.png']
copying  tmp_2504.13679/./F15F11_Sil.png to _build/html/
copying  tmp_2504.13679/./HD106906_fig1_paper.png to _build/html/
copying  tmp_2504.13679/./HD106906_fig2_contours.png to _build/html/
exported in  _build/html/2504.13679.md
    + _build/html/tmp_2504.13679/./F15F11_Sil.png
    + _build/html

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tblspc}{-0.1cm}$
$\newcommand{\rev}[1]{\textcolor{black}{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\farcs}{\hbox{.\!\!^{\prime\prime}}}$</div>



<div id="title">

# JWST's sharper view of EX Lup: cold water from ice sublimation during accretion outbursts

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.13377-b31b1b.svg)](https://arxiv.org/abs/2504.13377)<mark>Appeared on: 2025-04-21</mark> -  _Accepted for publication in ApJ Letters_

</div>
<div id="authors">

S. A. Smith, et al. -- incl., <mark>J. Bouwman</mark>

</div>
<div id="abstract">

**Abstract:** The unstable accretion phases during pre-main-sequence evolution of T Tauri stars produce variable irradiation and heating of planet-forming regions. A strong accretion outburst was observed with Spitzer-IRS in 2008 in EX Lup, the prototype of EXor variables, and found to increase the mid-infrared water and OH emission and decrease organic emission, suggesting large chemical changes. We present here two JWST-MIRI epochs of quiescent EX Lup in 2022 and 2023 obtained over a decade after the 2008 outburst and several months after a moderate burst in 2022. With JWST's sharper spectral view, we can now analyze water emission as a function of temperature in the two MIRI epochs and, approximately, also in the previous Spitzer epochs. This new analysis shows a strong cold water vapor "burst" in low-energy lines during the 2008 outburst, which we consider clear evidence for enhanced ice sublimation due to a recession of the snowline, as found in protostellar envelopes. JWST shows that EX Lup still has an unusually strong emission from cold water in comparison to other T Tauri disks, $\rev{suggesting $> 10$-yr-long freeze-out timescales in the inner disk surface}$ . EX Lup demonstrates that outbursts can significantly change the observed organic-to-water ratios and increase the cold water reservoir, providing chemical signatures to study the recent accretion history of disks. This study provides an unprecedented demonstration of the chemical evolution triggered by accretion outbursts in the Class II phase and of the high potential of time-domain experiments to reveal processes that may have fundamental implications on planet-forming bodies near the snowline.

</div>

<div id="div_fig1">

<img src="tmp_2504.13377/./EXLup_23umlines_ratios.png" alt="Fig1" width="100%"/>

**Figure 1. -** Coldest water detected in EX Lup, in comparison to the JDISCS sample from \citet{banzatti24}. The 23 $\mu$m line flux asymmetry diagnostic (the 1448/1615 K flux ratio) shows one of the largest values ever measured before, consistent with ice sublimation in the inner disk \citep[$\rev${the temperatures associated to specific 1448/1615 K values assume a column density of $N = 10^{17}$ cm$^{-2}$, see}][]{banzatti24}. The other object near EX Lup (JW2) is the young disk of IRAS 04385-2550. (*fig: 23umlines_ratios*)

</div>
<div id="div_fig2">

<img src="tmp_2504.13377/./all_spectra.png" alt="Fig7" width="100%"/>

**Figure 7. -** Mid-infrared spectral line variability observed with Spitzer-IRS \citep{banz12} and JWST-MIRI (this work) in the T Tauri disk of EX Lup over $\sim 20$ years (from 2005 with the SP1 epoch, to 2023 with the JW2 epoch). The two MIRI epochs are plotted in grey (JW1) and black (JW2) without offset, to demonstrate their similarity. The individual molecular models obtained for the JW2 epoch (Table \ref{tab: slab results}) are shown in different colors, their sum in grey. The same models are shown as downgraded to the resolving power of Spitzer-IRS for comparison to the SP1 and SP2 data, $\rev${with multiplication factors as labeled for each molecule;} additional models for water and OH are shown on top of the SP2 epoch to explain the more complex structure observed (see text for details). The vertical dashed lines show the range where the line flux of organic molecules is measured for Figure \ref{fig: IRS_correl}. $\rev${The lowest-energy lines covered by MIRI ($E_u$ = 878--1448 K), which are more sensitive to the snowline region at T $< 180 K$, are marked with a star (see also Figure \ref{fig: 23umlines_ratios}).} (*fig: all_spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2504.13377/./EXLup_diagnostic_diagram.png" alt="Fig8" width="100%"/>

**Figure 8. -** Water diagnostic diagrams from \citet{banzatti24}, with the JDISCS sample from that work shown for reference in light green. The position of the JW2 spectrum of EX Lup (large blue datapoint) in these diagrams shows: 1) a moderate slab radius ($\approx 0.3$ au) for the hot water component (left plot), 2) optically thick emission ($N \approx 10^{18}$ cm$^{-2}$) in the warm water component (middle plot), and 3) a temperature gradient with negative slope $\approx 0.45$--0.55 and with strong cold water enrichment (right plot, see also Figure \ref{fig: 23umlines_ratios}). These estimates are confirmed by the slab fit results both with discrete components and with the radial gradient (Sections \ref{sec: slab_model} and \ref{sec: cooler}). $\rev${The line ratios from the outburst SP2 spectrum are more uncertain due to line blending (see Appendix \ref{app: deblending}), but indicate a larger and colder water reservoir (see also Figure \ref{fig: 23umlines_ratios}).} (*fig: diagnostic_diagram*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.13377"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{\ensuremath{ {M}_\odot}}$
$\newcommand{\Rsun}{\ensuremath{ {R}_\odot}}$
$\newcommand{\Lsun}{\ensuremath{ {L}_\odot}}$
$\newcommand{\Teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\Mjup}{\ensuremath{ {M}_{\rm Jup}}}$
$\newcommand{\Rjup}{\ensuremath{ {R}_{\rm Jup}}}$
$\newcommand{\mic}{\ensuremath{{\mu{m}}} }$
$\newcommand{\Rearth}{\ensuremath{ {R}_{\bigoplus}}}$
$\newcommand{\Mearth}{\ensuremath{ {M}_{\bigoplus}}}$
$\newcommand{\Teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\safronov}{\ensuremath{\Theta}}$
$\newcommand{\as}{\ensuremath{^{\prime\prime}}}$
$\newcommand{\am}{\ensuremath{^\prime}}$
$\newcommand{\mc}[1]{\multicolumn{2}{c}{#1}}$
$\newcommand{\mcc}[1]{\multicolumn{3}{c}{#1}}$
$\newcommand{\er}[3]{\ensuremath{#1^{+#2}_{-#3}}}$
$\newcommand{\erc}[3]{\mc{\ensuremath{#1^{+#2}_{-#3}}}}$
$\newcommand{\erm}[3]{\mc{\ensuremath{#1^{+#2}_{-#3}}}}$
$\newcommand{\ercc}[3]{\mcc{\ensuremath{#1^{+#2}_{-#3}}}}$
$\newcommand{\ermcc}[5]{\mcc{\ensuremath{{#1 ^{+#2}_{-#3}} ^{+#4}_{-#5}}}}$
$\newcommand{\reff}[1]{{#1}}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\change}[1]{\textcolor{green}{#1}}$
$\newcommand{◦}{\hbox{^\circ}}$</div>



<div id="title">

# MIRI-JWST mid-infrared direct imaging of the debris disk of HD106906 : Structure and mass of the disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.13679-b31b1b.svg)](https://arxiv.org/abs/2504.13679)<mark>Appeared on: 2025-04-21</mark> - 

</div>
<div id="authors">

D. Rouan, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We report MIRI-JWST coronagraphic observations at 11.3 and 15.5 $\mic$ of the debris disk around the young star HD 106906. The wavelength range is sensitive to the thermal	  emission of the dust heated by the central star. The observations were made to characterize the structure of the disk through the thermal emission, to search for clues to the presence of a central void of dust particles, and to derive the mass of the dust and the temperature	  distribution. Another goal was also to constrain the size distribution of the grains. The data were reduced and calibrated  using the JWST pipeline. The  analysis was based on a forward-modeling of the images using a multiparameter radiative	  transfer model coupled to an optical code for coronagraphy processing. The disk is clearly detected at both wavelengths. The slight asymmetry is	  geometrically consistent with the asymmetry observed in the near-IR, but it is inconsistent the brightness distribution. The observed	  structure is well reproduced with a model of a disk (or belt) with a critical radius 70 au, a mildly	  inward-increasing density (index 2) and a steeper decrease outward (index -6).	This indication of a filled disk inside the critical radius is inconsistent with sculpting from an	  inner massive planet.	  The size distribution of the grains that cause the mid-IR emission is  well	  constrained by the flux ratio at the two wavelengths : 0.45 -- 10 $\mic$ and 0.65 -- 10 $\mic$ for silicate and graphite grains, respectively. The minimum size is consistent with predictions	  of blowout through radiative pressure. We derive a mass of the dust that causes the mid-IR emission of 3.3 -- 5.0 $10^{-3} \Mearth$ .	  When the larger grains (up to 1 cm) that cause the	  millimeter emission are included, we extrapolate this mass to 0.10 -- 0.16 $\Mearth$ .	  We point out to that this is fully consistent with ALMA observations of the disk	  in terms of dust mass and of its millimeter flux. We estimate the average dust temperature	  in the planetesimal belt to be 74 K, but the temperature range within the whole disk is rather wide: from 40 to 130 K.

</div>

<div id="div_fig1">

<img src="tmp_2504.13679/./F15F11_Sil.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
	   	Dependence of the ratio $R_f = F_{15.5} / F_{11.4}$ on the grain size
	   	distribution. The ratio $R_f$ is deduced from the model after integration
	   	of the flux in the considered size range.
	   	$R_f$  is plotted for a selection of different size ranges, all following a distribution in $a^{-3.5}$.
     Each segment depicts a range that is plotted in blue for silicate grains and in orange for graphite grains.
     The  dots correspond to single sizes of silicate grains.
	 The two retained ranges are indicated by thicker dash-dot segments.
     The  range of ratios deduced from observations is indicated by
        shaded purple. (*fig:F7*)

</div>
<div id="div_fig2">

<img src="tmp_2504.13679/./HD106906_fig1_paper.png" alt="Fig1" width="100%"/>

**Figure 1. -** Top: MIRI F1140C (left) and  F1550C (right)
	   	full-field coronagraphic images after data reduction (see text) and
	   	reference subtraction. The planet HD 106906 b can be seen at 7.3
	   	arcsec NW. Bottom: Same after zooming and binning by a factor 2.
	   	The orientation is standard (north is up, and east is to the left). The field of view is 22 $\times$ 22 arcsec$^2$ at the top and 11 $\times$ 11
	   	arcsec$^2$ at the bottom.  (*fig:F1*)

</div>
<div id="div_fig3">

<img src="tmp_2504.13679/./HD106906_fig2_contours.png" alt="Fig2" width="100%"/>

**Figure 2. -** MIRI 11.4 $\mu m$ coronagraphic image. The contours of
	   	the SPHERE H1-H2 image of Lagrange et al. (2015) are superimposed. (*fig:F2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.13679"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

448  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
